In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt


In [2]:
# import something
master_df = pd.read_csv('../datasets/MASTER_DF.csv')
master_df.head()

,entry,product,reacts,PubChem,SMILES,Mol,Fingerprint,dist,enzyme_class_1,enzyme_class_2,...,enzyme_class_7,n_C,n_H,n_O,n_N,n_P,n_S,n_X,DoU,MW
0,1.14.99.46,C20232,1.0,163311975,C/C(=C/NC(=O)N)/C(=O)OO,<rdkit.Chem.rdchem.Mol object at 0x1a7c6372b0>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.421512,1,0,...,0,5.0,8.0,4.0,2.0,0.0,0.0,0.0,3.0,160.129
1,3.5.1.110,C20232,1.0,163311975,C/C(=C/NC(=O)N)/C(=O)OO,<rdkit.Chem.rdchem.Mol object at 0x1a7c637300>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.467093,0,0,...,0,5.0,8.0,4.0,2.0,0.0,0.0,0.0,3.0,160.129
2,1.14.99.46,C00178,1.0,3478,CC1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637210>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.421512,1,0,...,0,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115
3,1.17.99.4,C00178,1.0,3478,CC1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637620>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.414081,1,0,...,0,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115
4,1.3.1.1,C00178,1.0,3478,CC1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637670>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.420926,1,0,...,0,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115


In [3]:
feature_df = master_df.iloc[:,7:24].copy()
feature_df.head()

,dist,enzyme_class_1,enzyme_class_2,enzyme_class_3,enzyme_class_4,enzyme_class_5,enzyme_class_6,enzyme_class_7,n_C,n_H,n_O,n_N,n_P,n_S,n_X,DoU,MW
0,0.421512,1,0,0,0,0,0,0,5.0,8.0,4.0,2.0,0.0,0.0,0.0,3.0,160.129
1,0.467093,0,0,1,0,0,0,0,5.0,8.0,4.0,2.0,0.0,0.0,0.0,3.0,160.129
2,0.421512,1,0,0,0,0,0,0,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115
3,0.414081,1,0,0,0,0,0,0,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115
4,0.420926,1,0,0,0,0,0,0,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115


In [4]:
features = np.array(feature_df) #shape balance array for regression
reactions = list(master_df['reacts'])

feature_train, feature_test, reaction_train, reaction_test = train_test_split(features, reactions,
                                                    test_size=0.20)

In [5]:
model_1 = linear_model.LogisticRegression(solver='liblinear', penalty='l1', random_state=1)
model_1.fit(feature_train, np.ravel(reaction_train))
predictions = model_1.predict(feature_test)
score = model_1.score(feature_test, reaction_test)
decision = model_1.decision_function(feature_test)
params = model_1.get_params()
pred_log = model_1.predict_log_proba(feature_test)
pred = model_1.predict_proba(feature_test)

score, pred, decision, model_1.classes_, model_1.coef_

(0.935124508519004, array([[0.81351044, 0.18648956],
        [0.95904498, 0.04095502],
        [0.69163838, 0.30836162],
        ...,
        [0.65790221, 0.34209779],
        [0.89621425, 0.10378575],
        [0.89881996, 0.10118004]]), array([-1.4729835 , -3.15346365, -0.80779005, ..., -0.65395966,
        -2.15585083, -2.18418127]), array([0., 1.]), array([[ 2.56679357e+01,  2.73740176e-02,  0.00000000e+00,
         -3.03928430e-01,  7.56800245e-01, -7.65589841e-01,
         -4.77612483e-02,  0.00000000e+00, -3.76285944e-02,
         -6.13927369e-02, -1.71146697e-01, -3.73727920e-02,
         -1.78920953e-01,  4.98290706e-02, -1.07706603e-01,
         -1.05327200e-01,  7.59915027e-03]]))

In [16]:
decision_distances = pd.Series(decision)
decision_distances.describe()

count    1526.000000
mean       -1.621057
std         3.884826
min       -10.499361
25%        -3.639795
50%        -2.688838
75%        -1.459687
max        14.685632
dtype: float64

In [23]:
prediction_values = pd.DataFrame(pred)
model_descriptive_df = pd.DataFrame()
model_descriptive_df['0']=prediction_values[0]
model_descriptive_df['1']=prediction_values[1]
model_descriptive_df['decision_distance']=decision_distances
model_descriptive_df

,0,1,decision_distance
0,0.813510,0.186490,-1.472983
1,0.959045,0.040955,-3.153464
2,0.691638,0.308362,-0.807790
3,0.994419,0.005581,-5.182827
4,0.999850,0.000150,-8.805832
5,0.886062,0.113938,-2.051134
6,0.953689,0.046311,-3.024947
7,0.985148,0.014852,-4.194677
8,0.974187,0.025813,-3.630734
9,0.969268,0.030732,-3.451240


In [6]:
updated = predictions.tolist()
confusion_matrix = confusion_matrix(reaction_test, updated)
print(confusion_matrix)
# upper left and lower right are correct: 1427
# lower left and upper right are incorrect : 99 

[[1184    5]
 [  94  243]]


In [7]:
print(classification_report(reaction_test, predictions))

              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      1189
         1.0       0.98      0.72      0.83       337

   micro avg       0.94      0.94      0.94      1526
   macro avg       0.95      0.86      0.90      1526
weighted avg       0.94      0.94      0.93      1526



______________
# linear regression on default data

logreg1=linear_model.LogisticRegression(random_state=1)        
logreg1.fit(feature_train, np.ravel(reaction_train)) #fit linear model, and shape default array for regression
score = logreg1.score(feature_test, reaction_test)
decision = logreg1.decision_function(feature_test)
params = logreg1.get_params()
pred_log = logreg1.predict_log_proba(feature_test)
pred = logreg1.predict_proba(feature_test)

score, pred, logreg1.classes_, logreg1.coef_ #np.ndarray.shape(pred)

#print('B0, B1: ',logreg.intercept_, logreg.coef_[0])

predictions1 = logreg1.predict(feature_test)
confusion_matrix = confusion_matrix(reaction_test, predictions1)
print(confusion_matrix)
# upper left and lower right are correct: 1400 for the first go
# lower left and upper right are incorrect : 111 for the first go

print(classification_report(reaction_test, predictions1))

logit_roc_auc = roc_auc_score(reaction_test, logreg1.predict(feature_test))
fpr, tpr, thresholds = roc_curve(reaction_test, logreg1.predict_proba(feature_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

plt.scatter(reaction_test, predictions1, alpha=0.1)
plt.xlabel('True value')
plt.ylabel('Predicted value')

logreg5 = linear_model.LogisticRegression(penalty='l1', random_state=1)
logreg5.fit(feature_train, np.ravel(reaction_train))
predictions5 = logreg5.predict(feature_test)
score = logreg5.score(feature_test, reaction_test)
decision = logreg5.decision_function(feature_test)
params = logreg5.get_params()
pred_log = logreg5.predict_log_proba(feature_test)
pred = logreg5.predict_proba(feature_test)

score, pred, logreg5.classes_, logreg5.coef_

print(classification_report(reaction_test, predictions5))

logit_roc_auc = roc_auc_score(reaction_test, logreg5.predict(feature_test))
fpr, tpr, thresholds = roc_curve(reaction_test, logreg5.predict_proba(feature_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

from sklearn.feature_selection import RFE



logreg2=linear_model.LogisticRegression() 
#logreg2.fit(feature_train, np.ravel(reaction_train),)
#predictions2 = logreg2.predict(feature_test)

#logreg = LogisticRegression()
rfe = RFE(logreg2, 17)
rfe = rfe.fit(feature_train, np.ravel(reaction_train))
pred_prob = rfe.predict_proba(feature_test)
rfe.support_, rfe.ranking_, pred_prob # rfe.feature_importances_
#print(rfe.support_)
#print(rfe.ranking_)
# looks like all of the features are important for prediction...

kf = KFold(n_splits=10, shuffle=True)
kf.get_n_splits(features, reactions)

int_reactions = [int(i) for i in reactions]


for train_index, test_index in kf.split(features, reactions):
    #print("TRAIN:", train_index, "TEST:", test_index)
    feature_train, feature_test = features[train_index], features[test_index]
    reaction_train, reaction_test = np.array(int_reactions)[train_index], np.array(int_reactions)[test_index]

    reg = linear_model.LogisticRegression().fit(feature_train, reaction_train)
    y_pred = reg.predict(feature_test)

 

print(classification_report(reaction_test, y_pred))

logit_roc_auc = roc_auc_score(reaction_test, reg.predict(feature_test))
fpr, tpr, thresholds = roc_curve(reaction_test, reg.predict_proba(feature_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

plt.scatter(reaction_test, y_pred, alpha=0.1)
plt.xlabel('True value')
plt.ylabel('Predicted value')

logreg3 = linear_model.LogisticRegression(solver='sag', max_iter=100000)
logreg3.fit(feature_train, np.ravel(reaction_train))
predictions3 = logreg3.predict(feature_test)

confusion_matrix = confusion_matrix(reaction_test, predictions3)
print(confusion_matrix)

print(classification_report(reaction_test, predictions3))

logit_roc_auc = roc_auc_score(reaction_test, logreg3.predict(feature_test))
fpr, tpr, thresholds = roc_curve(reaction_test, logreg3.predict_proba(feature_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

logreg4 = linear_model.LogisticRegression(solver='lbfgs', max_iter=10000)
logreg4.fit(feature_train, np.ravel(reaction_train))
predictions4 = logreg4.predict(feature_test)

logit_roc_auc = roc_auc_score(reaction_test, logreg4.predict(feature_test))
fpr, tpr, thresholds = roc_curve(reaction_test, logreg4.predict_proba(feature_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

print(classification_report(reaction_test, predictions4))